#### From time predictions to location predictions

We will try predicting the road condition in 1 meter intervals.

The issue before have been we have only used intervals, which just had the length of whatever far one could travel in 1 second. 

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
import h5py
from scipy.interpolate import CubicSpline
from copy import deepcopy
from tqdm import tqdm


In [2]:
parameter_dict = {
        'acc_long':     {'bstar': 198,      'rstar': 1,     'b': 198,   'r': 0.05   },
        'acc_trans':    {'bstar': 32768,    'rstar': 1,     'b': 32768, 'r': 0.04   },
        'acc_yaw':      {'bstar': 2047,     'rstar': 1,     'b': 2047,  'r': 0.1    },
        'brk_trq_elec': {'bstar': 4096,     'rstar': -1,    'b': 4098,  'r': -1     },
        'whl_trq_est':  {'bstar': 12800,    'rstar': 0.5,   'b': 12700, 'r': 1      },
        'trac_cons':    {'bstar': 80,       'rstar': 1,     'b': 79,    'r': 1      },
        'trip_cons':    {'bstar': 0,        'rstar': 0.1,   'b': 0,     'r': 1      }
    }

def convertdata(data, parameter):
    bstar = parameter['bstar']
    rstar = parameter['rstar']
    b = parameter['b']
    r = parameter['r']
    # We only convert data in the second column at idx 1 (wrt. 0-indexing), as the first column is time
    col0 = data[:,0]
    col1 = ((data[:,1]-bstar*rstar)-b)*r
    data = np.column_stack((col0, col1))
    return data


def unpack_hdf5(hdf5_file, convert: bool = False):
    with h5py.File(hdf5_file, 'r') as f:
        print(f)
        data = unpack_hdf5_(f, convert)
    return data


def unpack_hdf5_(group, convert: bool = False):
    data = {}
    for key in group.keys():
        if isinstance(group[key], h5py.Group):
            data[key] = unpack_hdf5_(group[key])
        else:
            if convert and key in parameter_dict:
                data[key] = convertdata(group[key][()], parameter_dict[key])
            else:
                d = group[key][()]
                if isinstance(d, bytes):
                    data[key] = d.decode('utf-8')
                else:
                    data[key] = group[key][()]
    return data


def find_best_start_and_end_indeces_by_lonlat(trip: np.ndarray, section: np.ndarray):
    # Find the start and end indeces of the section data that are closest to the trip data
    lon_a, lat_a = trip[:,0], trip[:,1]
    lon_b, lat_b = section[:,0], section[:,1]
    
    start_index = np.argmin(np.linalg.norm(np.column_stack((lon_a, lat_a)) - np.array([lon_b[0], lat_b[0]]), axis=1))
    end_index = np.argmin(np.linalg.norm(np.column_stack((lon_a, lat_a)) - np.array([lon_b[-1], lat_b[-1]]), axis=1))

    return start_index, end_index


def time_to_drive_X_meters(speed_kmh, distance_m=10):
    # Convert speed from km/h to m/s
    speed_ms = speed_kmh * (1000 / 3600)

    # Calculate time in seconds
    time_seconds = distance_m / speed_ms
    
    return time_seconds

In [3]:
autopi_hh = unpack_hdf5('../data/raw/AutoPi_CAN/platoon_CPH1_HH.hdf5')
gm_data = autopi_hh['GM']['16006']['pass_1'] # TODO choose a one true route. 
p79_hh = pd.read_csv('../data/raw/ref_data/cph1_zp_hh.csv', sep=';', encoding='unicode_escape')
p79_vh = pd.read_csv('../data/raw/ref_data/cph1_zp_vh.csv', sep=';', encoding='unicode_escape')

<HDF5 file "platoon_CPH1_HH.hdf5" (mode r)>


In [4]:
segments = h5py.File('../data/processed/segments.hdf5', 'r')        

In [5]:
segments.keys()

<KeysViewHDF5 ['0', '1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '2', '20', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '21', '210', '211', '212', '213', '214', '215', '216', '217', '218', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', 

In [8]:
segments["10"]

<HDF5 group "/10" (55 members)>

In [9]:
segments["10"].keys()

<KeysViewHDF5 ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '6', '7', '8', '9']>

In [10]:
segments.keys()
segments["10"].attrs["trip_name"]

'16006'

In [11]:
segments["10"]["10"].keys()

<KeysViewHDF5 ['aran', 'gm', 'p79']>

In [12]:
segments["10"]["10"]["gm"]

<HDF5 dataset "gm": shape (250, 42), type "<f8">

In [13]:
segments["10"]["10"]["gm"][:,15]

array([55.71061985, 55.71061963, 55.71061942, 55.7106192 , 55.71061898,
       55.71061875, 55.71061854, 55.71061832, 55.7106181 , 55.71061788,
       55.71061766, 55.71061744, 55.71061723, 55.71061701, 55.71061679,
       55.71061657, 55.71061635, 55.71061614, 55.71061592, 55.7106157 ,
       55.71061548, 55.71061526, 55.71061505, 55.71061483, 55.71061461,
       55.71061439, 55.71061418, 55.71061396, 55.71061374, 55.71061352,
       55.71061331, 55.71061309, 55.71061287, 55.71061266, 55.71061244,
       55.71061222, 55.710612  , 55.71061179, 55.71061157, 55.71061135,
       55.71061114, 55.71061092, 55.7106107 , 55.71061048, 55.71061027,
       55.71061005, 55.71060983, 55.71060962, 55.7106094 , 55.71060918,
       55.71060897, 55.71060875, 55.71060853, 55.71060832, 55.7106081 ,
       55.71060788, 55.71060767, 55.71060745, 55.71060723, 55.71060702,
       55.7106068 , 55.71060659, 55.71060637, 55.71060615, 55.71060594,
       55.71060572, 55.71060551, 55.71060529, 55.71060507, 55.71

In [14]:
len(segments["10"]["10"]["gm"][:,16])

250

In [15]:
len(segments["10"]["10"]["gm"][:,1])

250

In [16]:
# HH 
# Extract every 10th item starting from idx[0] to idx[1]+1, to get one location for each meter
lon_zp_hh = p79_hh['Lon']
lat_zp_hh = p79_hh['Lat']
idx_hh = find_best_start_and_end_indeces_by_lonlat(p79_hh[['Lat', 'Lon']].to_numpy(), gm_data['gps'][:,1:])
loc_hh_lon = lon_zp_hh[idx_hh[0]:idx_hh[1]+1:10]
loc_hh_lat = lat_zp_hh[idx_hh[0]:idx_hh[1]+1:10]

# Combine lon and lat into a list of lists
loc_hh = [[lon, lat] for lon, lat in zip(loc_hh_lon, loc_hh_lat)]

# For demonstration, let's print the first few pairs to verify
print(len(loc_hh))
print(loc_hh[:5])

1226
[[12.53019946, 55.7111808], [12.53018721, 55.71117492], [12.53017498, 55.71116905], [12.53016273, 55.71116317], [12.53015047, 55.7111573]]


In [17]:
# VH
# Extract every 10th item starting from idx[0] to idx[1]+1, to get one location for each meter
lon_zp_vh = p79_vh['Lon']
lat_zp_vh = p79_vh['Lat']
idx_vh = find_best_start_and_end_indeces_by_lonlat(p79_vh[['Lat', 'Lon']].to_numpy(), gm_data['gps'][:,1:])
loc_vh_lon = lon_zp_vh[idx_hh[0]:idx_vh[1]+1:10]
loc_vh_lat = lat_zp_vh[idx_hh[0]:idx_vh[1]+1:10]

# Combine lon and lat into a list of lists
loc_vh = [[lon, lat] for lon, lat in zip(loc_hh_lon, loc_hh_lat)]

# For demonstration, let's print the first few pairs to verify
print(len(loc_vh))
print(loc_vh[:5])

1226
[[12.53019946, 55.7111808], [12.53018721, 55.71117492], [12.53017498, 55.71116905], [12.53016273, 55.71116317], [12.53015047, 55.7111573]]


In [18]:
ex_mapping_to_the_two_best_seconds_for_each_trip = {0: {"16006": {"distance_segment_second_1": [100, 0, 0], "distance_segment_second_2": [100, 0, 0]},
                                                          "16008": {"distance_segment_second_1": [100, 0, 0], "distance_segment_second_2": [100, 0, 0]},
                                                          "16009": {"distance_segment_second_1": [100, 0, 0], "distance_segment_second_2": [100, 0, 0]},
                                                          "16010": {"distance_segment_second_1": [100, 0, 0], "distance_segment_second_2": [100, 0, 0]}, 
                                                          "16011": {"distance_segment_second_1": [100, 0, 0], "distance_segment_second_2": [100, 0, 0]}},
                                                    1: {"16006": {"distance_segment_second_1": [100, 0, 0], "distance_segment_second_2": [100, 0, 0]},
                                                          "16008": {"distance_segment_second_1": [100, 0, 0], "distance_segment_second_2": [100, 0, 0]},
                                                          "16009": {"distance_segment_second_1": [100, 0, 0], "distance_segment_second_2": [100, 0, 0]},
                                                          "16010": {"distance_segment_second_1": [100, 0, 0], "distance_segment_second_2": [100, 0, 0]}, 
                                                          "16011": {"distance_segment_second_1": [100, 0, 0], "distance_segment_second_2": [100, 0, 0]}}}
ex_mapping_to_the_two_best_seconds_for_each_trip[0]

{'16006': {'distance_segment_second_1': [100, 0, 0],
  'distance_segment_second_2': [100, 0, 0]},
 '16008': {'distance_segment_second_1': [100, 0, 0],
  'distance_segment_second_2': [100, 0, 0]},
 '16009': {'distance_segment_second_1': [100, 0, 0],
  'distance_segment_second_2': [100, 0, 0]},
 '16010': {'distance_segment_second_1': [100, 0, 0],
  'distance_segment_second_2': [100, 0, 0]},
 '16011': {'distance_segment_second_1': [100, 0, 0],
  'distance_segment_second_2': [100, 0, 0]}}

In [19]:
# Intialize the mapping dictionaty
all_trip_names = ['16006', '16008', '16009', '16010', '16011']
mapping_to_the_two_best_seconds_for_each_trip = {}

for index in range(len(loc_hh)):
    trip_data = {} 
    for trip_name in all_trip_names:
        # Each trip_name gets a dictionary with the two specified keys, each mapping to [100, 0, 0]
        trip_data[trip_name] = {
            "distance_segment_second_1": [100, 0, 0],
            "distance_segment_second_2": [200, 0, 0]
        }
        mapping_to_the_two_best_seconds_for_each_trip[index] = trip_data
mapping_to_the_two_best_seconds_for_each_trip[0]

{'16006': {'distance_segment_second_1': [100, 0, 0],
  'distance_segment_second_2': [200, 0, 0]},
 '16008': {'distance_segment_second_1': [100, 0, 0],
  'distance_segment_second_2': [200, 0, 0]},
 '16009': {'distance_segment_second_1': [100, 0, 0],
  'distance_segment_second_2': [200, 0, 0]},
 '16010': {'distance_segment_second_1': [100, 0, 0],
  'distance_segment_second_2': [200, 0, 0]},
 '16011': {'distance_segment_second_1': [100, 0, 0],
  'distance_segment_second_2': [200, 0, 0]}}

In [22]:
# Now fill out the dictionary with the correct values
for index, real_location in tqdm(enumerate(loc_hh), total=len(loc_hh), desc="Processing"):
    if index == 10:
        break 
    for segment in segments.keys():
        current_trip_name = segments[str(segment)].attrs["trip_name"]
        current_best_trip_1 = mapping_to_the_two_best_seconds_for_each_trip[index][current_trip_name]['distance_segment_second_1']
        current_best_trip_2 = mapping_to_the_two_best_seconds_for_each_trip[index][current_trip_name]['distance_segment_second_2']
        for second in segments[str(segment)].keys():
            current_second = segments[str(segment)][str(second)]
            current_second_lat = current_second["gm"][:,15]
            current_second_lon = current_second["gm"][:,16]
            current_second_locations = [[lon, lat] for lon, lat in zip(current_second_lon, current_second_lat)]
            closest_sample_arg = np.argmin(np.linalg.norm(np.column_stack((current_second_lon, current_second_lat)) - np.array([real_location[0], real_location[1]]), axis=1))
            best_at_second = current_second_locations[closest_sample_arg]
            distance = np.linalg.norm(np.array(best_at_second) - np.array(real_location))
            
            if distance < current_best_trip_1[0]:
                current_best_trip_2 = current_best_trip_1
                current_best_trip_1 = [distance, segment, second]
                mapping_to_the_two_best_seconds_for_each_trip[index][current_trip_name]['distance_segment_second_1'] = current_best_trip_1
                mapping_to_the_two_best_seconds_for_each_trip[index][current_trip_name]['distance_segment_second_2'] = current_best_trip_2
                
            elif distance < current_best_trip_2[0]:
                current_best_trip_2 = [distance, segment, second]
                mapping_to_the_two_best_seconds_for_each_trip[index][current_trip_name]['distance_segment_second_2'] = current_best_trip_2
            
            else: 
                continue


Processing:   1%|          | 10/1226 [01:03<2:08:49,  6.36s/it]


In [24]:
mapping_to_the_two_best_seconds_for_each_trip[0]

{'16006': {'distance_segment_second_1': [2.3317746094376914e-05, '0', '0'],
  'distance_segment_second_2': [2.3883416921457747e-05, '7', '1']},
 '16008': {'distance_segment_second_1': [1.0869581973696451e-05, '28', '2'],
  'distance_segment_second_2': [1.1523051548351325e-05, '33', '1']},
 '16009': {'distance_segment_second_1': [5.024776531263841e-06, '42', '2'],
  'distance_segment_second_2': [1.8639465280960233e-05, '38', '2']},
 '16010': {'distance_segment_second_1': [4.628884016767617e-06, '64', '11'],
  'distance_segment_second_2': [1.2269908010988185e-05, '64', '10']},
 '16011': {'distance_segment_second_1': [7.043385951903976e-07, '75', '11'],
  'distance_segment_second_2': [1.8116235160918723e-05, '69', '14']}}

True


In [ ]:
import pandas as pd

# mapping_to_the_two_best_seconds_for_each_trip is our dictionary
data = {
    'Trip ID': [],
    'Segment': [],
    'Distance Segment Second 1': [],
    'Distance Segment Second 2': []
}

for trip_id, segments in mapping_to_the_two_best_seconds_for_each_trip.items():
    for segment_id, values in segments.items():
        data['Trip ID'].append(trip_id)
        data['Segment'].append(segment_id)
        data['Distance Segment Second 1'].append(values['distance_segment_second_1'])
        data['Distance Segment Second 2'].append(values['distance_segment_second_2'])

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV
csv_filename = 'trip_data_hh.csv'
df.to_csv(csv_filename, index=False)


In [ ]:
hdf5_filename = 'trip_data_hh.hdf5'

# Create a new HDF5 file
with h5py.File(hdf5_filename, 'w') as hdf:
    for trip_id, segments in ex_mapping_to_the_two_best_seconds_for_each_trip.items():
        group = hdf.create_group(str(trip_id))
        for segment_id, segment_data in segments.items():
            # Convert each segment's data into a numpy array for easier handling
            ds_data = np.array([segment_data['distance_segment_second_1'],
                                segment_data['distance_segment_second_2']])
            # Each dataset is named after the segment ID
            group.create_dataset(segment_id, data=ds_data)

#### After mapping each AOI to 2 time windows for each car, we will predict the KPI of the AOI

There are in total 1226 AOIs. That is one every meter based on 

We have 10 windows for each AOI; 2 for each car and we have 5 cars.

In [7]:
# Use the speed to estimate the amount of sample points there are in the area of interest (AOI)
# The AOI is thought to be defined by an X meter long segment
A_speed = 50
B_speed = 30
C_speed = 40

sample_freq = 250  # Sample frequency in Hz
A_samples = time_to_drive_X_meters(A_speed) * sample_freq
B_samples = time_to_drive_X_meters(B_speed) * sample_freq
C_samples = time_to_drive_X_meters(C_speed) * sample_freq
total_samples = A_samples + B_samples + C_samples

A_weight = A_samples / total_samples
B_weight = B_samples / total_samples
C_weight = C_samples / total_samples


In [8]:
# We will now localise the point of interest in the time domains.
# It does not work to just take the first and last lon. and lat. and not the median either. 
# We need to be able to find the location fast, right now we will just 

# Should they all have a dictionary?
A_t1_data = [[1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10],
        [1,2,3,4,5,6,7,8,9,10]]
A_t1_pred = 5
